# 国民経済計算・長期経済統計の取得と可視化


In [ ]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path
# データ取得に用いるライブラリ
import json
import requests
# 前処理に用いるライブラリ
import lxml
import html5lib
import openpyxl
import pandas as pd
# 可視化に用いるライブラリ
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
from estat import (
    get_metainfo,
    get_statsdata,
    cleansing_statsdata,
    colname_to_japanese,
)

In [ ]:
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")
FRED_API_KEY = os.getenv("FRED_API_KEY")

In [ ]:
current_dir = Path.cwd()
data_path = (current_dir / "data" / "ch10").resolve()

## 国民経済計算とGDP

### 日本のGDP


In [ ]:
gdp_statsDataId = "0003109741"
gdp_meta = get_metainfo(appId, gdp_statsDataId)
gdp_metadata = gdp_meta["GET_META_INFO"]["METADATA_INF"]
gdp_total_num = gdp_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
gdp_total_num

In [ ]:
gdp_data = get_statsdata(appId, gdp_statsDataId)
gdp_value = colname_to_japanese(cleansing_statsdata(gdp_data))
gdp_value.columns

In [ ]:
gdp_value = gdp_value.assign(**{
    "年": gdp_value["時間軸（四半期）コード"].str[:4],
    "月": gdp_value["時間軸（四半期）コード"].str[-2:],
}).assign(**{
    "年月": lambda df: pd.to_datetime(df["年"] + "-" + df["月"] + "-01"),
    "系列名": lambda df: df["国内総生産_名目原系列コード"]
    + "_"
    + df["国内総生産_名目原系列"],
})

In [ ]:
gdp_value.loc[gdp_value["時間軸（四半期）コード"]=="2023001012", ["系列名", "値"]]

In [ ]:
sectors = ["12_民間最終消費支出", "22_財貨・サービス_輸出", "23_財貨・サービス_輸入",
    "29_<参考>民間需要", "30_<参考>公的需要"]
gdp_cond = gdp_value["系列名"].isin(sectors)
gdp_cond &= gdp_value["年"].astype(int) <= 2024  # 年の期間を絞る
gdp_long_df = gdp_value.loc[gdp_cond, ["年月", "系列名", "値"]]
gdp_wide_df = gdp_long_df.pivot(columns="系列名", index="年月", values="値")
# 家計消費支出と企業投資を計算
# 公的需要、企業投資、家計消費支出の順に並ぶよう便宜的に33,34の系列番号を採番
gdp_wide_df = gdp_wide_df.assign(**{
    "23_財貨・サービス_輸入": -gdp_wide_df["23_財貨・サービス_輸入"],  # 輸入はマイナスに変換
    "29_投資": gdp_wide_df["29_<参考>民間需要"] - gdp_wide_df["12_民間最終消費支出"]
})
gdp_cols = ["22_財貨・サービス_輸出", "23_財貨・サービス_輸入", 
    "12_民間最終消費支出", "29_投資", "30_<参考>公的需要"]
gdp_df = gdp_wide_df[gdp_cols]
# 年次(Year End)形式に変換。なお、四半期で可視化する場合は以下のコードは不要
gdp_df = gdp_df[gdp_cols].groupby(pd.Grouper(freq="YE")).sum()
gdp_df.tail()

In [ ]:
# 整然データ形式に変換
gdp_tidy_df = gdp_df.stack().reset_index().rename(columns={0: "値"})
# GDP合計を計算
gdp = gdp_df.sum(axis=1)
gdp_fig = go.Figure()
# 各GDPの内訳を棒グラフで追加
for c in gdp_cols:
    gdp_fig.add_trace(go.Bar(
        x=gdp_tidy_df.loc[gdp_tidy_df["系列名"] == c, "年月"],
        y=gdp_tidy_df.loc[gdp_tidy_df["系列名"] == c, "値"],
        name=c
    ))
# GDP合計を折れ線グラフで追加
gdp_fig.add_trace(go.Scatter(x=gdp.index, y=gdp, name="GDP", mode="lines+markers"))
# y軸と棒の積み上げのレイアウトを調整
gdp_fig.update_layout(width=1000, height=600, font={"size": 18},
    xaxis={"ticksuffix": "年"},
    yaxis={"tickformat": ",.0f", "ticksuffix": "(十億円)"},
    barmode="relative",  # 輸入はマイナスのためrelativeで積み上げ
)
gdp_fig.show()

### 世界のGDP

#### 世界銀行のGDPデータ


In [ ]:
wb_api_url = "https://api.worldbank.org/v2"
indicators_url = f"{wb_api_url}/indicators"
indicators_params = {"per_page": 50000, "format": "json"}
indicators_res = requests.get(indicators_url, indicators_params)
indicators_json = indicators_res.json()
indicators_data = indicators_json[1]
indicators_df = pd.DataFrame(indicators_data)
name_match = r"gdp.*US\$"
wb_cond = indicators_df["name"].str.contains(name_match, case=False)
wb_cond &= indicators_df["sourceNote"] != ""
indicators_df.loc[wb_cond, ["id", "name", "sourceNote"]].tail()

In [ ]:
indicator = "NY.GDP.MKTP.CD"
countries_list = ["US", "CN", "JP", "IN", "GB", "DE", "FR"
    , "CA", "RU", "KR", "IT", "AU", "BR", "MX", "ID"]
countries = ";".join(countries_list)
wb_url = f"{wb_api_url}/countries/{countries}/indicators/{indicator}"
wb_start = 2005
wb_end = 2024
wb_date = f"{wb_start}:{wb_end}"
wb_params = {
    "date": wb_date,
    "per_page": "25000",
    "format": "json",
}
wb_res = requests.get(wb_url, wb_params)
wb_json = wb_res.json()
wb_df = pd.DataFrame(wb_json[1])
wb_df = wb_df.assign(**{
    "indicator_id": [x["id"] for x in wb_df["indicator"]],
    "indicator_value": [x["value"] for x in wb_df["indicator"]],
    "country_id": [x["id"] for x in wb_df["country"]],
    "国": [x["value"] for x in wb_df["country"]],
    "年": wb_df["date"].astype(int),
    "GDP": wb_df["value"] // 1_000_000_000  # 10億単位に変換
})
wb_df.iloc[-5:, 2:]

In [ ]:
wb_colors = px.colors.qualitative.Plotly + px.colors.qualitative.Set3
wb_fig = px.line(
    wb_df, x="年", y="GDP", color="国", color_discrete_sequence=wb_colors
)
wb_fig.update_layout(width=900, height=650, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "十億ドル", "title": None},
)
wb_fig.show()

#### GapminderのGDPデータ


In [ ]:
gapminder_df = px.data.gapminder()
gapminder_df.head()

In [ ]:
gapminder_fig = px.scatter(gapminder_df,
    x="gdpPercap", 
    y="lifeExp",
    color="continent", 
    size="pop",
    animation_frame="year",
    animation_group="country",
    hover_name="country",
    log_x=True,
    size_max=55,
    range_x=[100, 100_000],
    range_y=[25, 90],
)
gapminder_fig.show()

## 物価とインフレ率

### 日本の物価とインフレ率


In [ ]:
cpi_statsDataId = "0003427113"
cpi_meta = get_metainfo(appId, cpi_statsDataId)
cpi_metadata = cpi_meta["GET_META_INFO"]["METADATA_INF"]
cpi_total_num = cpi_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
cpi_total_num

In [ ]:
cpi_params = {"cdCat01": "0001", "cdArea": "00000", "cdTimeFrom": "1970000101"}
cpi_data = get_statsdata(appId, cpi_statsDataId, params=cpi_params)
cpi_value = colname_to_japanese(cleansing_statsdata(cpi_data))
cpi_value.columns

In [ ]:
cpi_value = cpi_value.assign(**{
    "年": cpi_value["時間軸（年・月）コード"].str[:4],
    "月": cpi_value["時間軸（年・月）コード"].str[-2:]  # 年・年度の'00'が存在
})
# 時間軸（年・月）階層レベルは年月
cpi_cond = cpi_value["表章項目"] == "指数"
cpi_cond &= cpi_value["時間軸（年・月）階層レベル"] == "4"  # 月次
cpi_df_monthly = cpi_value.loc[cpi_cond]
# 年月の列を作成
cpi_df_monthly = cpi_df_monthly.assign(**{
    "年月": pd.to_datetime(cpi_df_monthly["年"] + "-" + cpi_df_monthly["月"] + "-01")
})
cpi = cpi_df_monthly[["年月", "2020年基準品目", "値"]]
cpi.head()

In [ ]:
inflation_cond = cpi_value["表章項目"] == "前月比・前年比・前年度比"
inflation_cond &= cpi_value["時間軸（年・月）階層レベル"] == "1"
inflation_rate = cpi_value.loc[inflation_cond, ["年", "2020年基準品目", "値"]]
inflation_rate = inflation_rate.assign(**{
    "年": inflation_rate["年"].astype(int),
    "値": inflation_rate["値"] / 100
})

In [ ]:
# サブプロットの描画領域を作成
cpi_fig = make_subplots(
    rows=1, cols=2, subplot_titles=("物価(2020基準品目)", "インフレ率(2020基準品目)")
)
# CPIの折れ線グラフを描画領域に追加
cpi_fig.add_trace(
    go.Scatter(x=cpi["年月"], y=cpi["値"], mode="lines", name="物価"), row=1, col=1
)
# インフレ率の折れ線グラフを描画領域に追加
cpi_fig.add_trace(
    go.Scatter(
        x=inflation_rate["年"], y=inflation_rate["値"], mode="lines", name="インフレ率"
    ),
    row=1,
    col=2,
)
# X軸とグラフのレイアウト調整
cpi_fig.update_xaxes(ticksuffix="年", row=1)
cpi_fig.update_layout(width=900, height=500, font={"size": 18})
cpi_fig.show()

### 米国の物価とインフレ率


In [ ]:
fred_url = "https://api.stlouisfed.org/fred/series/observations"
observation_start = "1980-01-01"
observation_end = "2024-12-31"
fred_params = {
    "api_key": FRED_API_KEY,
    "file_type": "json",
    "observation_start": observation_start,
    "observation_end": observation_end,
    "frequency": "m",  # Monthly
    "aggregation_method": "eop",  # End of period
    "series_id": "CPIAUCNS"
}
fred_res = requests.get(fred_url, params=fred_params)
us_cpi = pd.DataFrame(fred_res.json()["observations"]).assign(**{
    "date": lambda df: pd.to_datetime(df["date"]),
    "value": lambda df: pd.to_numeric(df["value"], errors="coerce"),
})
us_cpi

In [ ]:
bls_headers = {"Content-type": "application/json"}
bls_data = json.dumps(
    {"seriesid": ["CUSR0000SA0"], "startyear": "2020", "endyear": "2024"}
)
bls_res = requests.post(
    "https://api.bls.gov/publicAPI/v2/timeseries/data/",
    data=bls_data,
    headers=bls_headers,
)
bls_json = json.loads(bls_res.text)
bls_cpi = pd.DataFrame(bls_json["Results"]["series"][0]["data"])
bls_cpi.head()

In [ ]:
jpus_fred_param = fred_params.copy()
jpus_fred_param["series_id"] = "DEXJPUS"
jpus_fred_res = requests.get(fred_url, params=jpus_fred_param)
dexjpus = pd.DataFrame(jpus_fred_res.json()["observations"]).assign(**{
    "date": lambda df: pd.to_datetime(df["date"]),
    "value": lambda df: pd.to_numeric(df["value"], errors="coerce"),
})
dexjpus

In [ ]:
us_cpi = us_cpi.assign(**{"指標": "米国の物価"})
dexjpus = dexjpus.assign(**{"指標": "ドル円レート"})
fred_df = pd.concat([us_cpi, dexjpus])[["date", "value", "指標"]]
fred_wide = fred_df.pivot(columns="指標", index="date", values="value")
cpi_jp = cpi.rename(columns={"値": "日本の物価"})[["年月", "日本の物価"]].set_index("年月")
merged_cpi = (fred_wide
    .merge(cpi_jp, left_index=True, right_index=True, how="inner")
    .rename_axis("年月")
)
initial_rate = merged_cpi["ドル円レート"].iloc[0]
initial_ratio = merged_cpi["日本の物価"].iloc[0] / merged_cpi["米国の物価"].iloc[0]
ppp_df = merged_cpi.assign(**{
    "購買力平価": merged_cpi["日本の物価"] / merged_cpi["米国の物価"]
                * (initial_rate / initial_ratio)
})
ppp_df.tail()

In [ ]:
value_vars = ["ドル円レート", "購買力平価"]
ppp_tidy = pd.melt(ppp_df[value_vars],
    value_vars=value_vars,
    var_name="レート",
    value_name="円/ドル",
    ignore_index=False,
).reset_index()
ppp_fig = px.line(ppp_tidy, x="年月", y="円/ドル", color="レート")
ppp_fig.update_layout(width=900, height=500, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
)
ppp_fig.show()

### 【コードのみ】状態空間モデル

In [ ]:
!uv pip install statsmodels

In [ ]:
import statsmodels.api as sm

In [ ]:
local_level_model_season = sm.tsa.UnobservedComponents(
    endog=us_cpi["value"], 
    level=True, 
    stochastic_level=True,
    trend=True,
    stochastic_trend=True,
    seasonal=12,
    stochastic_seasonal=True
)
res_local_level_season = local_level_model_season.fit()
#print(res_local_level_season.summary())
fig_local_level_season = res_local_level_season.plot_components()

In [ ]:
trend = pd.Series(res_local_level_season.trend['smoothed'], index=us_cpi["date"]).reset_index()
trend.columns = ["date", "trend"]
trend_fig = px.line(trend, x="date", y="trend")
trend_fig.show()

## 経済指数

### 景気動向指数


In [ ]:
di_statsDataId = "0003446461"
di_meta = get_metainfo(appId, di_statsDataId)
di_metadata = di_meta["GET_META_INFO"]["METADATA_INF"]
di_total_num = di_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
di_total_num

In [ ]:
di_data = get_statsdata(appId, di_statsDataId)
di_value = colname_to_japanese(cleansing_statsdata(di_data))
di_value.columns

In [ ]:
di_value = di_value.assign(**{
    "年": di_value["時間軸(月次)コード"].str[:4],
    "月": di_value["時間軸(月次)コード"].str[-2:],
}).assign(**{
    "年月": lambda df: pd.to_datetime(
        df["年"].astype(str) + df["月"].astype(str), format="%Y%m"
    )
})
di_cols = ["年月", "先行指数・一致指数・遅行指数", "値"]
di_df = di_value.loc[di_value["表章項目"] == "CI指数", di_cols]

In [ ]:
di_fig = px.line(di_df, x="年月", y="値", color="先行指数・一致指数・遅行指数")
di_fig.update_layout(width=900, height=500, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None}, 
    yaxis={"title": None}
)
di_fig.show()

### 鉱工業指数・第3次産業指数


In [ ]:
!pip install openpyxl

In [ ]:
file_name = "b2020_202506s.xlsx"
workbook = openpyxl.load_workbook(data_path / file_name)
sheet_names = workbook.sheetnames
print("シート数: ", len(sheet_names), "\nシート一覧: ", sheet_names)
workbook.close()

In [ ]:
iip_dfs = []
for n in range(32):  # 事前に確認したシート数
    # Excelのn番目のシートをデータフレームで読み込み
    # 日本語のヘッダーと値の始まる直前の行(7)をheaderに指定
    iip_xls = pd.read_excel(
        data_path / file_name, sheet_name=n, header=[0, 2, 4, 6, 7], engine="openpyxl"
    ).dropna(how="all")
    new_col = iip_xls.columns[1][0]
    print(f"{n+1}シート目: ", new_col)
    # colの1行目は「鉱工業」等の産業、最後の行は英語行のため除外し、"_"つないだ新しい列名に変換
    iip_xls.columns = [
        "_".join([str(c) for c in col[1:-1] if "Unnamed" not in c])
        for col in iip_xls.columns
    ]
    iip_xls["産業"] = new_col
    iip_dfs.append(iip_xls)
iip_df = pd.concat(iip_dfs)
iip_df.iloc[3:8, :5]

In [ ]:
iip_df = iip_df.assign(**{
    "時間軸コード": iip_df["時間軸コード"].astype(int).astype(str),
}).assign(**{
    "年": lambda df: df["時間軸コード"].str[:4],
    "月": lambda df: df["時間軸コード"].str[-2:]
})
iip_cond = (iip_df["月"]!="00") & (iip_df["時間軸コード"].str[-4:-2]==iip_df["月"])
iip_cols = ["年", "月", "産業", "生産_季節調整済指数_指数値"]
iip = iip_df.loc[iip_cond, iip_cols]
iip = iip.assign(**{"年月": iip["年"] + iip["月"]})

In [ ]:
iip_cols = ["産業", "生産_季節調整済指数_指数値", "年月"]
yy = iip["年"].max()
mm = iip[iip["年"] == yy]["月"].max()
slope_df = iip.loc[iip["年月"].str[-2:] == mm, iip_cols]
industries = sorted(slope_df["産業"].unique())
colors = px.colors.qualitative.Plotly + px.colors.qualitative.Dark24
industry_color_map = {industry: color for industry, color in zip(industries, colors)}

iip_fig = px.line(slope_df,
    x="年月",
    y="生産_季節調整済指数_指数値",
    color="産業",
    color_discrete_map=industry_color_map,
    markers=True,
)

for industry in industries:
    value = slope_df[(slope_df["産業"] == industry) & (slope_df["年月"] == f"{yy}{mm}")]
    iip_fig.add_annotation(
        x=1.05,
        y=value["生産_季節調整済指数_指数値"].values[0],
        text=industry,
        showarrow=False,
        xanchor="left",
        yanchor="middle",
        font={"size": 10, "color": industry_color_map[industry]},
    )

iip_fig.update_layout(width=800, height=1000, font={"size": 18},
    xaxis={"title": None},
    yaxis={"title": None},
    showlegend=False,
    title={"text": "鉱工業指数スロープチャート", "x": 0.5, "xanchor": "center"},
)
iip_fig.show()

## 貿易統計

In [ ]:
ts_statsDataId = "0003313965"
ts_meta = get_metainfo(appId, ts_statsDataId)
ts_metadata = ts_meta["GET_META_INFO"]["METADATA_INF"]
ts_total_num = ts_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
ts_total_num

In [ ]:
!uv pip install lxml html5lib

In [ ]:
html_path = "https://www.customs.go.jp/tariff/2025_04_01/data/print_j_10.htm"
hs_df = pd.read_html(html_path, na_values="", encoding="cp932")[2]
col_lv0, col_lv1 = "統計番号Statisticalcode", "番号H.S.code"
prefix = "関税率Tariffrate"
tariff_suffixes = ["基本General", "暫定Temporary", "WTO協定WTO", 
    "特恵GSP", "特別特恵LDC", "日米貿易協定US"]
tariff_cols = ["統計品目番号", f"{col_lv0}{col_lv1}", col_lv0, "品名Description"] + \
    [f"{prefix}{s}" for s in tariff_suffixes] + \
    ["関税率（経済連携協定）Tariffrate(EPA)RCEP(中国)"]
hs_rice_us = (hs_df.set_axis([
    (c0 if c0 == c1 else c0 + ('' if str(c1).startswith('Unnamed') else str(c1)))
    .replace(" ", "").replace("◆1", "")
    for c0, c1 in hs_df.columns], axis=1)
    .assign(**{"統計品目番号": lambda df: (
        df[f"{col_lv0}{col_lv1}"].ffill().astype(str) + 
        df[col_lv0].fillna("").astype(str)
    ).str.replace(".", "")})
    [tariff_cols].iloc[65:75, :])
hs_rice_us

In [ ]:
na_csv_path = "https://www.customs.go.jp/toukei/suii/html/data/d42ca003.csv"
na_df = pd.read_csv(na_csv_path, header=[2, 3], encoding="cp932")
na_df.columns = [("" if "Unnamed" in j else j) + i for i, j in na_df.columns]
na_df.head().T

In [ ]:
area_dict = {
    "アジア": "d42ca001",
    "太平洋": "d42ca002",
    "北米": "d42ca003",
    "中南米": "d42ca004",
    "西欧": "d42ca005",
    "中東欧・ロシア等": "d42ca006",
    "中東": "d42ca007",
    "アフリカ": "d42ca008",
    "特殊地域": "d42ca009",
}
def format_columns_assign_area(df, area):
    return df.set_axis(
        [("" if "Unnamed" in j else j) + i for i, j in df.columns],
        axis=1
    ).assign(**{"地域": [area] * len(df)})
trade_dfs = []
for k in area_dict:
    filename = area_dict[k]
    na_csv_path = f"https://www.customs.go.jp/toukei/suii/html/data/{filename}.csv"
    tmp_df = pd.read_csv(na_csv_path, header=[2, 3], encoding="cp932").pipe(
        lambda df: format_columns_assign_area(df, k)
    )
    #tmp_df.to_csv(f"ch10_輸出入額_{k}.csv")
    trade_dfs.append(tmp_df[["Years", "Exp-Total", "Imp-Total", "地域"]])
trade_df = pd.concat(trade_dfs)
trade_df.head()

In [ ]:
trade_df["Imp-Total"] = -trade_df["Imp-Total"]
trade_tidy = pd.melt(trade_df,
    id_vars=["Years", "地域"],
    value_vars=["Exp-Total", "Imp-Total"],
    var_name="輸出・輸入",
    value_name="輸出入額",
)
trade_tidy = trade_tidy.assign(**{
    "輸出入額": trade_tidy["輸出入額"] / 1_000_000,
    "地域輸出入": trade_tidy["地域"] + trade_tidy["輸出・輸入"]
})

In [ ]:
trade_fig = px.line(
    trade_tidy, x="Years", y="輸出入額", line_group="地域輸出入", color="地域"
)
trade_fig.update_layout(width=900, height=500, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "百万円", "title": None},
    title={"text": "輸出額(プラス)と輸入額(マイナス)", "x": 0.5, "xanchor": "center"},
)
trade_fig.show()

## 国際収支統計


In [ ]:
# URLから直接読み込みも可能
bp_path = (
    "https://www.mof.go.jp/policy/international_policy/reference/balance_of_payments/"
    "bp_trend/bpnet/sbp/s-1/6s-1-1.csv"
)
# エンコーディングはcp932としCSVファイルを読み込む
bp_trend = pd.read_csv(bp_path, skiprows=24, header=list(range(9)), encoding="cp932", thousands=",")
# 読み込んだデータフレームの列名がマルチインデックスのためシングルインデックスの列名を変換
bp_trend = bp_trend.set_axis(
    ["（暦年）", "半期", "西暦", "half-year",
    "経常収支", "貿易・サービス収支", "貿易収支", "輸出", "輸入", "サービス収支",
    "第一次所得収支", "第二次所得収支", "資本移転等収支", 
    "金融収支", "直接投資", "証券投資", "金融派生商品", "その他投資",
    "外貨準備", "誤差脱漏"],
    axis=1,
)
# 西暦は「2022C.Y.」形式で半期ごとにNullがあるためffillで補完し、整数型に変換
bp_trend = bp_trend.assign(**{
    "西暦": bp_trend["西暦"].ffill().str.replace("C.Y.", "").str[:4].astype(int)
})
# 西暦ごとに集計するため、不要な列を削除
bop_df = bp_trend.drop(["（暦年）", "半期", "half-year"], axis=1)
# 西暦ごとに各収支金額を合計
bop = bop_df.groupby("西暦").sum()
bop.tail().T

In [ ]:
bop_fig = make_subplots()
# 棒グラフと折れ線グラフに追加するデータカラム名を用意
bar_traces = ["貿易収支", "サービス収支", "第一次所得収支", "第二次所得収支"]
scatter_traces = ["経常収支", "金融収支", "資本移転等収支"]
# 棒グラフを追加
for n, col in enumerate(bar_traces):
    bop_fig.add_trace(go.Bar(
        x=bop.index,
        y=bop[col],
        name=col,
        marker_color=px.colors.qualitative.Set3[n],
    ))
# 折れ線グラフを追加
for n, col in enumerate(scatter_traces):
    bop_fig.add_trace(go.Scatter(
        x=bop.index,
        y=bop[col],
        name=col,
        mode="lines+markers",
        line={"color": px.colors.qualitative.Plotly[n]},
    ))
bop_fig.update_layout(width=1000, height=600, font={"size": 18},
    barmode="relative",
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "百万円", "title": None},
)
bop_fig.show()